In [71]:
import torch
import pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

import json

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [45]:
business_file = "yelp_dataset/yelp_academic_dataset_business.json"

categories = {}

# Get busines_id and categories
with open(business_file, 'r', encoding='utf-8') as f:
    for line in f:
        # Load JSON data
        business = json.loads(line)
        
        if business['categories'] is not None:
            categories[business['business_id']] = business['categories']

In [46]:
reviews_file = "yelp_dataset/yelp_academic_dataset_review.json"

reviews = {}

# Get business_id and reviews
with open(reviews_file, 'r', encoding='utf-8') as f:
    for line in f:
        # Load JSON data
        review = json.loads(line)
        
        reviews[review['business_id']] = review['text'] # normalizing

In [47]:
pred_dict = {}

for biz_id, cats in categories.items():
    for c in cats.split(','):
        if c not in pred_dict.keys():
            pred_dict[c] = []
        pred_dict[c].append(reviews[biz_id])

In [48]:
len(pred_dict)

2454

Decrease the reviews per category by 50%, if one left, delete category. Also decrease categories by 50%

In [49]:
c = int(len(pred_dict.keys()) * 0.5)
to_delete = list(pred_dict.keys())[0: c]

for k in to_delete:
    pred_dict.pop(k)

In [50]:
len(pred_dict)

1227

In [51]:
temp = pred_dict.copy();
for category, reviews in temp.items():
    n = len(reviews)
    if n== 3:
        pred_dict.pop(category)
    pred_dict[category] = reviews[0: int(n/2)]

In [52]:
# embeddings = model.encode(pred_dict['Bubble Tea'])
X = []
Y = []

for x, y in pred_dict.items():
    X.append(model.encode(y))
    Y.append(x)

# #Print the embeddings
# for sentence, embedding in zip(pred_dict['Bubble Tea'], embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

In [53]:

# #Print the embeddings
# for sentence, embedding in zip(pred_dict['Bubble Tea'], embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

KeyError: 'Bubble Tea'

The data has already been processed into a pkl file for ease:

In [57]:
if False:
    with open('Sentences.pkl', 'wb') as f:
        pickle.dump(X, f)
    
    with open('Labels.pkl', 'wb') as r:
        pickle.dump(Y, r)

if True:
    with open('Sentences.pkl', 'rb') as f:
        X = pickle.load(f)
    with open('Labels.pkl', 'rb') as r:
        Y = pickle.load(r)

In [78]:
place_holder = zip(X, Y)
X_updated = []
Y_updated = []

for sentences, label in place_holder:
    for sentence in sentences:
        s = np.array(sentence)
        X_updated.append(s)
        Y_updated.append(label)

In [79]:
features = np.array(X_updated)
features.shape

(10645, 384)

In [80]:
labels = np.array(Y_updated)
labels.shape

(10645,)

In [81]:
enc = OneHotEncoder()

In [82]:
enc.fit(labels.reshape(-1, 1))
labels = enc.transform(labels.reshape(-1, 1))

In [83]:
labels.shape

(10645, 1073)

In [84]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [85]:
y_train

<8516x1073 sparse matrix of type '<class 'numpy.float64'>'
	with 8516 stored elements in Compressed Sparse Row format>

In [86]:
X_train = torch.tensor(X_train, dtype=torch.float32).to(torch.int64)
X_test = torch.tensor(X_test, dtype=torch.float32).to(torch.int64)
y_train = torch.tensor(y_train.todense(), dtype=torch.float32)
y_test = torch.tensor(y_test.todense(), dtype=torch.float32)

In [87]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=4)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=True, num_workers=4)

In [88]:
class CategoryPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CategoryPredictor, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x, dim=1)  # Average pooling over the sequence length
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [98]:
class run_model:
    def __init__(self, input_size, hidden_size, output_size, train_loader, test_loader):
        self.model = CategoryPredictor(input_size, hidden_size, output_size)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.train_loader = train_loader
        self.test_loader = test_loader


    def run(self, epochs):
        num_epochs = epochs
        for epoch in range(num_epochs):
            print("Start:", epoch)
            for small_x, small_y in self.train_loader:
                self.optimizer.zero_grad()
                print("Start Forward Pass:", epoch)
                outputs = self.model(small_x)
                print("Finished Forward Pass:", epoch)
                loss = self.criterion(outputs, torch.argmax(small_y, dim=1))
                print("Starting Backpropagation:", epoch)
                loss.backward()
                print("Finished Backpropagation:", epoch)
                self.optimizer.step()
            print("Epoch Fin:", epoch)

        print("Done")

    def evaluate(self, X_test_tfidf, y_test):
        with torch.no_grad():
            self.model.eval()
            predictions = self.model(X_test_tfidf)
            _, predicted_labels = torch.max(predictions, 1)
            return accuracy_score(torch.argmax(y_test, dim=1), predicted_labels.numpy())

In [99]:
input_size = X_train.shape[0]
hidden_size = 50 
output_size = labels.shape[1]

model = run_model(input_size, hidden_size, output_size, train_loader, test_loader)
model.run(3)

Start: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backpropagation: 0
Finished Backpropagation: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backpropagation: 0
Finished Backpropagation: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backpropagation: 0
Finished Backpropagation: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backpropagation: 0
Finished Backpropagation: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backpropagation: 0
Finished Backpropagation: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backpropagation: 0
Finished Backpropagation: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backpropagation: 0
Finished Backpropagation: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backpropagation: 0
Finished Backpropagation: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backpropagation: 0
Finished Backpropagation: 0
Start Forward Pass: 0
Finished Forward Pass: 0
Starting Backprop

In [100]:
accuracy = model.evaluate(X_test, y_test)

In [101]:
accuracy

0.010803193987787695